In [ ]:
import typing
import copy
import math
import random

"""
XOR using neat
"""

""" spotkanie 9 listopad
    1) dokończyć swoją implementację
    2) gymnasium + NEAT
    3) zastąpić sumowanie filtrem kalmana
"""
#class Genome:
#    class Node:
#        types = ['Input', 'Hidden', 'Output', 'Sensor']
#        def __init__(self, type:tuple[str],num:int ,  active=True):
#            self.inType= type[0]
#            self.outType= type[1]
#            self.num = num
#        
#        def __str__(self):
#            return f"Node {self.inType} to {self.outType}. Number: {self.num}"
#        def __repr__(self):
#            return self.__str__()
#
#    class Connection:
#        def __init__(self,input:int, out:int, weight:float,enabled:bool, innov:int):
#            self.enabled= enabled
#            self.input = input
#            self.output = out
#            self.weight = weight
#            self.innov = innov
#
#        def split_in_two(self, newNode:int, innov:int):
#            in_conn = Genome.Connection(input=self.input, out=newNode, weight=1, enabled=True, innov= innov+1)
#            out_conn = Genome.Connection(input=newNode, out=self.output, weight=self.weight, enabled=True, innov= innov+2)
#            self.enabled = False
#            return [in_conn, out_conn]
#
#        def __str__(self):
#            return f"Connection from {self.input} to {self.output}. Status: {self.enabled}"
#
#        def __repr__(self):
#            return self.__str__()
#
#    def __init__(self):
#        self.node_genes : dict[int, Genome.Node] = {}
#        self.connection_genes : list[Genome.Connection] = []
#
#    def clone(self):
#        return copy.deepcopy(self)
#
#    def mutate_add_node(self):
#        random_connection = self.connection_genes[random.randint(0, len(self.connection_genes)-1)]
#        new_node_init = (
#            self.node_genes[random_connection.input].outType,
#            self.node_genes[random_connection.output].inType
#            )
#        node_innov = max(self.node_genes.keys()) + 1
#        self.node_genes[node_innov] = Genome.Node(new_node_init, num=node_innov)
#        conn_innov = len(self.connection_genes) - 1
#        new_conns = random_connection.split_in_two(newNode=node_innov, innov=conn_innov )
#        self.connection_genes += new_conns
#
#    def mutate_add_connection(self):
#        choices_in =  list(filter(None,[i if i.inType != 'Output' else None for i in  self.node_genes.values()]))
#        choices_out = list(filter(None,[i if i.inType != 'Input' else None for i in  self.node_genes.values()]))
#        choice_in = random.choice(choices_in)
#        choice_out = random.choice(filter(lambda i : i != choice_in, choices_out))
#        innov = len(self.connection_genes) -1
#        new_conn = Genome.Connection(input=choice_in, out=choice_out, weight=random.random(), enabled=True, innov=innov)
#        self.connection_genes.append(new_conn)
#
#    def connect_nodes(self, idx1, idx2):
#        new_conn = Genome.Connection(input=idx1, out=idx2, weight=1, enabled=True, innov=len(self.connection_genes))
#        self.connection_genes.append(new_conn)
#    
#    def __str__(self):
#        return f"Genome with {len(self.connection_genes)} connections and {len(self.node_genes.keys())} nodes\n {list(map(lambda x:x.__str__(),self.node_genes.values()))} \n {list(map(lambda x:x.__str__(), self.connection_genes))}]"
#
#    def __repr__(self):
#        return self.__str__()
#
#    def add_node(self, node):
#        i = self.get_current_node_num() + 1 
#        self.node_genes[i] = node
#        return i
#    
#    def get_current_node_num(self) -> int:
#        return max(self.node_genes.keys() or [0])
#
#
#    @staticmethod
#    def init_with_shape(shape:tuple[int]):
#        g = Genome()
#        input_size, output_size = shape
#        inputs = []
#        outputs = []
#
#        # construct input nodes
#        for i in range(input_size):
#            n = Genome.Node(("Sensor", "Input"), num=g.get_current_node_num()+1 )
#            g.add_node(n)
#            inputs.append(n)
#
#        # construct output nodes
#        for o in range(output_size):
#            n = Genome.Node(("Input", "Output"), num=g.get_current_node_num()+1)
#            g.add_node(n)
#            outputs.append(n)
#
#        # connect inputs with outputs densely
#        for i in inputs:
#            for o in outputs:
#                g.connect_nodes(i, o)
#        return g

In [3]:
# New approach

import random

def relu(x):
    if x > 0:
        return x
    else:
        return 0

WEIGHT_MUTATION_CHANCE = 0.8
NEW_CONNECTION_CHANCE = 0.05
NEW_NODE_CHANCE = 0.03
ACTIVATION_FUNC = relu

C1 = 1.0
C2 = 1.0
C3 = 0.4
SPECIES_THRESHOLD = 3.0  # Threshold for determining if two genomes belong to the same species


In [4]:

class Node:
    def __init__(self, node_id, node_type):
        self.node_id = node_id
        self.node_type = node_type  #  'input' | 'hidden' | 'output'

    def __repr__(self):
        return f"{self.node_type} Node {self.node_id} "


In [5]:

class Connection:
    def __init__(self, in_node, out_node, weight, enabled=True, innov_number=None):
        self.in_node = in_node
        self.out_node = out_node
        self.weight = weight
        self.enabled = enabled
        self.innov_number = innov_number  # crossover

    def __repr__(self):
        return f"Conn from {self.in_node} to {self.out_node}, weight:{self.weight}, en:{self.enabled}, innv:{self.innov_number}"


In [6]:

class Species:
    def __init__(self, representative):
        self.members = [representative]
        self.representative = representative

    def add_member(self, genome):
        self.members.append(genome)

    def clear_members(self):
        self.members = []

    def adjust_fitnesses(self):
        total_fitness = sum([genome.fitness for genome in self.members])
        for genome in self.members:
            genome.fitness /= total_fitness

    def set_representative(self):
        self.representative = random.choice(self.members)


def speciate(genomes, species):
    for spec in species:
        spec.clear_members()

    for genome in genomes:
        found_species = False
        for spec in species:
            if genome.compatibility_distance(spec.representative) < SPECIES_THRESHOLD:
                spec.add_member(genome)
                found_species = True
                break
        if not found_species:
            new_species = Species(genome)
            species.append(new_species)

    # Remove empty species
    species = [spec for spec in species if spec.members]

    return species


In [8]:

class Genome:
    def __init__(self):
        self.nodes = []
        self.connections = []
        
        self.next_node_id = 0
        self.next_innov_number = 0
    
    @staticmethod
    def create_from_shape(shape:tuple[int,int]):
        inputs,outputs = shape
        g = Genome()

        for i in range(inputs):
            g.add_node('input')

        for o in range(outputs):
            g.add_node('output')

        ins = list(filter(lambda x:x.node_type == 'input', g.nodes))
        outs = list(filter(lambda x:x.node_type == 'output', g.nodes))
        for i in ins:
            for o in outs:
                g.add_connection(i.node_id, o.node_id, 1.0)
        return g

    def add_node(self, node_type):
        node = Node(self.next_node_id, node_type)
        self.nodes.append(node)
        self.next_node_id += 1

    def evaluate(self, input):
        ff_in = {}
        for (i, n) in enumerate(input):
            ff_in[i] = n
        return feed_forward(self, ff_in, ACTIVATION_FUNC)

    def add_connection(self, in_node, out_node, weight):
        connection = Connection(in_node, out_node, weight, innov_number=self.next_innov_number)
        self.connections.append(connection)
        self.next_innov_number += 1

    def mutate(self):
        for connection in self.connections:
            if random.random() < WEIGHT_MUTATION_CHANCE:
                connection.weight += random.uniform(-1, 1) 

        if random.random() < NEW_CONNECTION_CHANCE:
            node1 = random.choice(self.nodes)
            node2 = random.choice(self.nodes)
            if node1.node_type != 'output' and node2.node_type != 'input' and node1 != node2:
                self.add_connection(node1.node_id, node2.node_id, random.uniform(-1, 1))

        if random.random() < NEW_NODE_CHANCE:
            connection = random.choice(self.connections)
            connection.enabled = False

            new_node = self.add_node('hidden')
            self.add_connection(connection.in_node, new_node.node_id, 1.0)
            self.add_connection(new_node.node_id, connection.out_node, connection.weight)


In [9]:

import numpy as np
def feed_forward(genome, input_values:dict[int,float], activation_func):
    all_output_nodes = set([n for n in genome.nodes if n.node_type == 'output'])
    valid_connections = [ c for c in genome.connections if c.enabled] 
    print('input values', input_values)
    print('genome', genome)
    grouped_by_output_node = {}
    values = input_values 
    for c in valid_connections:
        if grouped_by_output_node.get(c.out_node):
            grouped_by_output_node[c.out_node].add(c)
        else:
            grouped_by_output_node[c.out_node] = set([c])
    print('after init, ', grouped_by_output_node)
    def eval_node(node_id):
        print('eval called with node_id', node_id, 'and values:', values, 'and gbon:', grouped_by_output_node)
        if(values.get(node_id)):
            return values.get(node_id)
        else:
            deps = []
            for conn in grouped_by_output_node[node_id]:
                deps.append(eval_node(conn.in_node) * conn.weight)
            v = activation_func(sum(deps))
            values[node_id] = v
            return v
    
    return [eval_node(o.node_id) for o in all_output_nodes]


In [10]:

def compatibility_distance(self, other_genome):
        matching_weight_diffs = []
        disjoint_count = 0
        excess_count = 0

        self_genes = {c.innov_number: c for c in self.connections}
        other_genes = {c.innov_number: c for c in other_genome.connections}

        for innov in set(self_genes.keys()).union(other_genes.keys()):
            if innov in self_genes and innov in other_genes:
                matching_weight_diffs.append(abs(self_genes[innov].weight - other_genes[innov].weight))
            elif innov in self_genes:
                if innov > max(other_genes.keys()):
                    excess_count += 1
                else:
                    disjoint_count += 1
            elif innov in other_genes:
                if innov > max(self_genes.keys()):
                    excess_count += 1
                else:
                    disjoint_count += 1

        avg_weight_diff = sum(matching_weight_diffs) / len(matching_weight_diffs) if matching_weight_diffs else 0.0
        N = max(len(self.connections), len(other_genome.connections))
        N = max(N, 1)

        distance = (C1 * disjoint_count + C2 * excess_count) / N + C3 * avg_weight_diff
        return distance



In [57]:
INPUT = [[0,0], [0,1], [1,0], [1,1]]
OUTPUT = [0, 1, 1, 0]
import copy
def eval(genome):
    result = list(map(lambda l: l[0], [feed_forward(genome, input_values= dict(enumerate(iv)), activation_func=ACTIVATION_FUNC) for iv in INPUT]))
    assert(len(OUTPUT) == len(result))
    error = sum([ r - o for (r,o) in zip(result, OUTPUT)])
    y = (4 - error) ** 2
    return y

class Population:
    population = 40
    max_iters = 100
    def __init__(self, shape):
        self.generation = 0
        self.genomes = []
        default = Genome.create_from_shape(shape)
        for _ in range(self.population):
            self.genomes.append(copy.deepcopy(default))
        self.species = [self.genomes[0]]

    def run(self):
        self.generation += 1
        results = map(eval, self.genomes)
        top_results = sorted(results, reverse=True)
        print(f"generation {self.generation}, top 5 results are {top_results[0:5]}\n")
        for (g,r) in zip(self.genomes, results): g.fitness = r 


    def run_all(self):
        while self.generation < self.max_iters: 
            self.run()


generation 1, top 5 results are [4.0, 4.0, 4.0, 4.0, 4.0]



In [44]:
%debug

> /var/folders/92/3gzjc7p14mvb4pqkx2ryk0680000gn/T/ipykernel_78483/2037887146.py(8)<listcomp>()
      6     assert(len(OUTPUT) == len(result))
      7 
----> 8     error = sum([ r - o for (r,o) in zip(result, OUTPUT)])
      9     y = (4 - error) ** 2
     10     print('results from eval', y)

[0, 1, 1, 0]
*** NameError: name 'result' is not defined
*** NameError: name 'result' is not defined
[[0, 0], [0, 1], [1, 0], [1, 1]]
*** NameError: name 'genome' is not defined
*** NameError: name 'error' is not defined
*** NameError: name 'y' is not defined
*** NameError: name 'self' is not defined
*** NameError: name 'self' is not defined
--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user


In [42]:

import numpy as np
def feed_forward(genome:Genome, input_values:dict[int,float], activation_func):
    """
    Second approach. First make whole dependency tree, then evaluate it recursively
    """
    all_output_nodes = set([n for n in genome.nodes if n.node_type == 'output'])
    valid_connections = [ c for c in genome.connections if c.enabled] 
    dependency_tree = {}
    values = input_values 

    for c in valid_connections:
        if dependency_tree.get(c.out_node) is not None:
            dependency_tree[c.out_node].add(c)
        else:
            dependency_tree[c.out_node] = set([c])

    def eval_node(node_id):
        if(values.get(node_id) is not None):
            return values.get(node_id)
        else:
            deps = []
            dependants = dependency_tree.get(node_id)
            assert(dependants is not None) 
            for conn in dependants:
                deps.append(eval_node(conn.in_node) * conn.weight)
            v = activation_func(sum(deps))
            values[node_id] = v
            return v
    
    return [eval_node(o.node_id) for o in all_output_nodes]


In [50]:
p = Population((2,1))
p.run()

[[0], [1.0], [1.0], [2.0]]


TypeError: unsupported operand type(s) for -: 'list' and 'int'

In [ ]:
class Problem:
    def __init__(self, inputs, outputs, compute_fitness):
        if len(inputs) != len(outputs): raise Exception("Number of inputs and outputs dont match")


In [ ]:

gen = Genome()
gen.add_node('input') #     0
gen.add_node('input') #     1
gen.add_node('hidden')#     2
gen.add_node('output')#     3
gen.add_node('output')#     4
gen.add_connection(0,4, weight=0.8)
gen.add_connection(1,2, weight=0.8)
gen.add_connection(0,3, weight=0.8)
gen.add_connection(1,4, weight=0.8)
gen.add_connection(2,3, weight=0.8)
result = feed_forward(gen, {0:0.25, 1:0.5}, relu)
print("RESULT:", result)
